<a href="https://colab.research.google.com/github/denis-kasak/lstm-activities/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import os
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
import torch.nn.functional as F
import multiprocessing
import requests

def download_file_if_not_exists(url="https://raw.githubusercontent.com/denis-kasak/lstm-activities/main/data.txt", file_path="/content/data.txt"):
    if not os.path.exists(file_path):
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"File downloaded and saved to {file_path}")
        else:
            print(f"Failed to download file from {url}")
    else:
        print(f"File already exists at {file_path}")


def read_data(file_path):
    """
    Read the data from the file and return a pandas DataFrame.
    """
    # Initialize an empty list to store the data
    data = []

    # Open the file and read each line
    with open(file_path, "r") as file:
        for line in file:
            # Split the line by comma and then strip the semicolon at the end
            split_line = line.strip().split(",")
            if split_line:
                # Remove the semicolon from the last element
                split_line[-1] = split_line[-1].replace(";", "")
                data.append(split_line)

    # Create a DataFrame from the list
    dataframe = pd.DataFrame(
        data, columns=["user", "activity", "timestamp", "x", "y", "z"]
    )

    # Convert appropriate columns to numeric types
    dataframe["user"] = pd.to_numeric(dataframe["user"], errors="coerce")
    dataframe["timestamp"] = pd.to_numeric(dataframe["timestamp"], errors="coerce")
    dataframe["x"] = pd.to_numeric(dataframe["x"], errors="coerce")
    dataframe["y"] = pd.to_numeric(dataframe["y"], errors="coerce")
    dataframe["z"] = pd.to_numeric(dataframe["z"], errors="coerce")

    return dataframe


def preprocess_data(dataframe, window_size, test_split, seed, batch_size):
    """
    Perform preprocessing on the DataFrame.
    """

    # Drop unused columns
    dataframe.dropna(inplace=True)

    # Normalize the accelerometer columns
    scaler = StandardScaler()
    dataframe[["x", "y", "z"]] = scaler.fit_transform(dataframe[["x", "y", "z"]])

    def create_segments(df, window_size, batch_size):
        # Slide a "window_size" wide window with a step size of 1
        segments = []
        labels = []
        for i in range(0, len(df) - window_size, batch_size):
            xs = df["x"].values[i : i + window_size]
            ys = df["y"].values[i : i + window_size]
            zs = df["z"].values[i : i + window_size]
            # Retrieve the most often used label in this segment
            label = stats.mode(df["activity"][i : i + window_size])[0]
            segments.append([xs, ys, zs])
            labels.append(label)
        return segments, labels

    label_encoder = LabelEncoder()
    dataframe["activity"] = label_encoder.fit_transform(dataframe["activity"])

    segments, labels = create_segments(dataframe, window_size, batch_size)

    # Reshape segments and labels
    reshaped_segments = np.asarray(segments, dtype=np.float32).reshape(
        -1, window_size, 3
    )

    labels = np.asarray(labels, dtype=np.int64)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        reshaped_segments, labels, test_size=test_split, random_state=seed
    )

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

    test_data = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    return train_loader, test_loader


class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # Modify this part to include dropout in LSTM layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate if num_layers > 1 else 0.0)

        # Add a dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Forward propagate through the LSTM layers
        x, _ = self.lstm(x, (h0, c0))

        # Apply dropout to the output of the last LSTM layer
        x = self.dropout(x)

        # Apply softmax to the output of the last time step
        x = self.fc(x[:, -1, :])
        return F.softmax(x, dim=1)


def compile_and_train(model: LSTMModel, train_loader, num_epochs, learning_rate, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # print(f"Epoch {epoch+1}, Loss: {loss.item()}")

    return model


def evaluate(model, test_loader, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct, total


def train_model(num_epochs):
    batch_size = 1024
    window_size = 200
    test_split = 0.2
    seed = 42
    input_dim = 3
    hidden_dim = 64
    output_dim = 6
    num_layers = 2
    num_epochs = num_epochs
    learning_rate = 0.0025
    dropout_rate = 0.3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    data_frame = read_data("data.txt")

    train_loader, test_loader = preprocess_data(
        data_frame, window_size, test_split, seed, batch_size
    )

    model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers, dropout_rate).to(device)

    model = compile_and_train(model, train_loader, num_epochs, learning_rate, device)

    correct, total = evaluate(model, test_loader, device)

    accuracy = correct / total

    model_path = os.path.join(
        "/", "content",
        "models", f"model_{format(num_epochs, '04d')}_epochs_{accuracy}_accuracy.pt"
    )
    torch.save(model.state_dict(), model_path)


def main():
    for i in range(10, 101, 10):
        print(f"Training model with {i} epochs.")
        train_model(i)
    for i in range(150, 501, 50):
        print(f"Training model with {i} epochs.")
        train_model(i)
    for i in range(500, 1001, 100):
        print(f"Training model with {i} epochs.")
        train_model(i)
    for i in range(1000, 5001, 500):
        print(f"Training model with {i} epochs.")
        train_model(i)
    for i in range(5000, 10001, 1000):
        print(f"Training model with {i} epochs.")
        train_model(i)


download_file_if_not_exists()
main()


KeyboardInterrupt: 